<a href="https://colab.research.google.com/github/jaya-shankar/DSS_project/blob/main/model_build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## setting up enviroment

In [1]:
!pip install tensorflow_decision_forests
!pip install wurlitzer

     |████████████████████████████████| 13.4 MB 4.6 MB/s 
     |████████████████████████████████| 462 kB 48.4 MB/s 


In [2]:
!rm -rf DSS_project
!git clone https://github.com/jaya-shankar/DSS_project.git

Cloning into 'DSS_project'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 182 (delta 87), reused 110 (delta 39), pack-reused 0
Receiving objects: 100% (182/182), 29.68 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [3]:
import pandas as pd
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split

In [4]:
root_path = "/content/DSS_project/datasets"
raw_path = "/content/DSS_project/raw_datasets"


## fetching dataset

In [17]:
df = pd.read_csv(root_path+"/final_combined_df.csv")
df.drop(columns=['Unnamed: 0'],inplace=True)
df

,state_code,date,cases,deaths,lat,long,no_of_hospitals,miles_of_road,percent_democrat,population,...,above_65,percent_white,persons_per_household,percent_in_poverty,avg_wind_speed,avg_temp,people_fully_vaccinated,CurrentHospitalizations,CurrentlyInICU,CurrentlyOnVentilator
0,WA,2021-01-12,2292.0,76.0,47.411715,-120.592492,102,81022.0,60.330912,15410562,...,15.4,68.0,2.55,10.3,15.03,40.76,17689.0,978.0,0.0,111.0
1,WA,2021-01-13,2327.0,40.0,47.411715,-120.592492,102,81022.0,60.330912,15410562,...,15.4,68.0,2.55,10.3,15.03,40.76,22625.0,979.0,0.0,93.0
2,WA,2021-01-14,2437.0,40.0,47.411715,-120.592492,102,81022.0,60.330912,15410562,...,15.4,68.0,2.55,10.3,15.03,40.76,0.0,983.0,0.0,93.0
3,WA,2021-01-15,2161.0,23.0,47.411715,-120.592492,102,81022.0,60.330912,15410562,...,15.4,68.0,2.55,10.3,15.03,40.76,31581.0,966.0,0.0,105.0
4,WA,2021-01-17,2400.0,0.0,47.411715,-120.592492,102,81022.0,60.330912,15410562,...,15.4,68.0,2.55,10.3,15.03,40.76,0.0,889.0,0.0,158.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22340,WV,2022-04-04,171.0,4.0,38.642579,-80.613729,59,38878.0,30.112471,3587432,...,19.9,92.1,2.42,17.8,18.72,41.61,1025162.0,141.0,39.0,20.0
22341,WV,2022-04-06,107.0,25.0,38.642579,-80.613729,59,38878.0,30.112471,3587432,...,19.9,92.1,2.42,17.8,18.72,41.61,1026240.0,127.0,34.0,19.0
22342,WV,2022-04-07,87.0,8.0,38.642579,-80.613729,59,38878.0,30.112471,3587432,...,19.9,92.1,2.42,17.8,18.72,41.61,1026528.0,125.0,35.0,20.0
22343,WV,2022-04-08,92.0,4.0,38.642579,-80.613729,59,38878.0,30.112471,3587432,...,19.9,92.1,2.42,17.8,18.72,41.61,1026847.0,116.0,33.0,16.0


## Normalizing the dataset

In [18]:
no_normalization_columns = {'date','state_code','lat','long'}
columns = list(df.columns)
for column in columns:
  if column in no_normalization_columns:
    continue
  df[column] = (df[column] - df[column].min())/(df[column].max() - df[column].min())
df

,state_code,date,cases,deaths,lat,long,no_of_hospitals,miles_of_road,percent_democrat,population,...,above_65,percent_white,persons_per_household,percent_in_poverty,avg_wind_speed,avg_temp,people_fully_vaccinated,CurrentHospitalizations,CurrentlyInICU,CurrentlyOnVentilator
0,WA,2021-01-12,0.010054,0.029699,47.411715,-120.592492,0.162455,0.252362,0.488364,0.182961,...,0.483146,0.648876,0.486726,0.223140,0.114409,0.404552,0.000628,0.044972,0.000000,0.120521
1,WA,2021-01-13,0.010207,0.015631,47.411715,-120.592492,0.162455,0.252362,0.488364,0.182961,...,0.483146,0.648876,0.486726,0.223140,0.114409,0.404552,0.000803,0.045018,0.000000,0.100977
2,WA,2021-01-14,0.010690,0.015631,47.411715,-120.592492,0.162455,0.252362,0.488364,0.182961,...,0.483146,0.648876,0.486726,0.223140,0.114409,0.404552,0.000000,0.045202,0.000000,0.100977
3,WA,2021-01-15,0.009479,0.008988,47.411715,-120.592492,0.162455,0.252362,0.488364,0.182961,...,0.483146,0.648876,0.486726,0.223140,0.114409,0.404552,0.001121,0.044420,0.000000,0.114007
4,WA,2021-01-17,0.010528,0.000000,47.411715,-120.592492,0.162455,0.252362,0.488364,0.182961,...,0.483146,0.648876,0.486726,0.223140,0.114409,0.404552,0.000000,0.040879,0.000000,0.171553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22340,WV,2022-04-04,0.000750,0.001563,38.642579,-80.613729,0.084838,0.118593,0.038616,0.031233,...,0.988764,0.987360,0.371681,0.842975,0.313546,0.425355,0.036380,0.006484,0.008035,0.021716
22341,WV,2022-04-06,0.000469,0.009769,38.642579,-80.613729,0.084838,0.118593,0.038616,0.031233,...,0.988764,0.987360,0.371681,0.842975,0.313546,0.425355,0.036418,0.005840,0.007005,0.020630
22342,WV,2022-04-07,0.000382,0.003126,38.642579,-80.613729,0.084838,0.118593,0.038616,0.031233,...,0.988764,0.987360,0.371681,0.842975,0.313546,0.425355,0.036429,0.005748,0.007211,0.021716
22343,WV,2022-04-08,0.000404,0.001563,38.642579,-80.613729,0.084838,0.118593,0.038616,0.031233,...,0.988764,0.987360,0.371681,0.842975,0.313546,0.425355,0.036440,0.005334,0.006799,0.017372


In [19]:
df.columns

Index(['state_code', 'date', 'cases', 'deaths', 'lat', 'long',
       'no_of_hospitals', 'miles_of_road', 'percent_democrat', 'population',
       'avg_income', 'mean_travel_time', 'above_65', 'percent_white',
       'persons_per_household', 'percent_in_poverty', 'avg_wind_speed',
       'avg_temp', 'people_fully_vaccinated', 'CurrentHospitalizations',
       'CurrentlyInICU', 'CurrentlyOnVentilator'],
      dtype='object')

## Assigning Importance

In [ ]:
decision_parameters = {
                      "cases" : 0,
                     "deaths" : 0,
    "CurrentHospitalizations" : 0,
             "CurrentlyInICU" : 0,
      "CurrentlyOnVentilator" : 0,
    "people_fully_vaccinated" : 0,
            "no_of_hospitals" : 0,

                 "population" : 0,
      "persons_per_household" : 0,
                   "above_65" : 0,

                 "avg_income" : 0,
         "percent_in_poverty" : 0,

           "mean_travel_time" : 0,

              "miles_of_road" : 0,

             "avg_wind_speed" : 0,
                   "avg_temp" : 0,

           "percent_democrat" : 0,

              "percent_white" : 0,    
}